# DATA EXTRACTION AND CLEANING

## Carga de bibliotecas necesarias

In [ ]:
# Instal·lem tot allò necessari
try:
    from google.cloud import bigquery
except ImportError:
    !pip install google-cloud-bigquery
    !pip install google-cloud
    !pip install db-dtypes

In [ ]:
# Importar librerías requeridas
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
import google
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import iplot
from google.cloud import bigquery
from google.oauth2 import service_account
import plotly.io as pio
pio.renderers.default='notebook'

# Librerías adicionales para modelos de machine learning
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [5]:
# Coloque aca la ruta de su archivo .json
credentials = service_account.Credentials.from_service_account_file(
r"/content/mimic.json")

# Create a BigQuery client
client = bigquery.Client(credentials=credentials)

# Definir el dataset y las tablas
# hosp_dataset_id = 'mimiciv_hosp'
icu_dataset_id = 'mimiciv_icu'
#----------------------
admissions_table = 'caregiver'
chartevents_table = 'chartevents'
d_items_table = 'd_items'
datetimeevents_table = 'datetimeevents'
icustay_table = 'icustay'
ingredientevents_table = 'ingredientevents'
inputevents_table = 'inputevents'
outputevents_table = 'outputevents'
procedureevents_table = 'procedureevents'

## Extracción de tablas

### Tabla de gente que SÍ necesitó ventilación mecánica

In [ ]:
# Extrae eventos clínicos de chartevent con el itemid que interesa, une las medidas con el nombre de d_item para tener una label
query = f"""
SELECT
  ce.subject_id,
  ce.hadm_id,
  ce.stay_id,
  ce.charttime,
  ce.itemid,
  di.label,
  ce.valuenum
FROM
  `mimiciv_icu.chartevents` ce
JOIN
  `mimiciv_icu.d_items` di
ON
  ce.itemid = di.itemid
WHERE
  ce.itemid IN (
    220210,
    220227, 220277,
    223830,
    220235,
    220955, 224828,
    227443, 220739, 223900, 223901, 220045, 220050, 220051, 223762, 226228
  )
"""

# Convierte el resultado anterior en un DataFrame de Panda
all_df = client.query(query).to_dataframe()

#
all_df['charttime'] = pd.to_datetime(all_df['charttime'])

In [ ]:
# Consulta de procedimientos con ventilación
query_vent = f"""
SELECT
  subject_id,
  hadm_id,
  stay_id,
  starttime,
  endtime,
  ordercategoryname
FROM
  `mimiciv_icu.procedureevents`
WHERE
  LOWER(ordercategoryname) LIKE '%ventilation%'
"""
pe_vent_df = client.query(query_vent).to_dataframe()

# Asegurar que las fechas estén en formato datetime
pe_vent_df['starttime'] = pd.to_datetime(pe_vent_df['starttime'])


In [ ]:
# Unir todos los datos y filtrar vectorizadamente
# Filtrar solo los subject_id/stay_id necesarios
unique_stays = pe_vent_df[['subject_id', 'stay_id']].drop_duplicates()
filtered_df = all_df.merge(unique_stays, on=['subject_id', 'stay_id'])

# Unir con eventos de ventilación
merged_df = filtered_df.merge(
    pe_vent_df[['subject_id', 'stay_id', 'starttime']],
    on=['subject_id', 'stay_id']
)

# Filtrar solo las filas dentro de la ventana de 6h antes
merged_df = merged_df[
    (merged_df['charttime'] >= merged_df['starttime'] - pd.Timedelta(hours=6)) &
    (merged_df['charttime'] <= merged_df['starttime'])
]

# Renombrar
merged_df = merged_df.rename(columns={'starttime': 'vent_starttime'})

In [ ]:
# Concatenar todo en un solo DataFrame
vent_df = merged_df.copy()

# Pivotear para formato tabla
si_vent_df = vent_df.pivot_table(
    index=['subject_id', 'hadm_id', 'stay_id', 'charttime', 'vent_starttime'],
    columns='itemid',
    values='valuenum',
    aggfunc='first'
).reset_index()

In [ ]:
# Renombrar columnas con nombres legibles
item_names = {
    220210: 'RR',
    220227: 'SaO2',
    220277: 'SpO2',
    223830: 'PH_arterial',
    220235: 'PCO2_arterial',
    224828: 'Base_Excess',
    227443: 'HCO3_serum',
    220739: 'GCS_Eye_Opening',
    223900: 'GCS_Verbal_Response',
    223901: 'GCS_Motor_Response',
    220045: 'Heart_Rate',
    220050: 'BP_Systolic',
    220051: 'BP_Diastolic',
    223762: 'Temperature_Celsius',
    226228: 'Gender',
}
si_vent_df = si_vent_df.rename(columns=item_names)

# Guardar resultado
si_vent_df.to_csv('si_vent_6h.csv', index=False)

In [ ]:
pd.read_csv('si_vent_6h.csv')

### Tabla de gente que NO necesitó ventilación mecánica

In [ ]:
# Obtener fecha de ingreso al ICU
query_stay = f"""
SELECT
  subject_id,
  stay_id,
  intime
FROM
  `mimiciv_icu.icustays`
"""
stay_df = client.query(query_stay).to_dataframe()
stay_df['intime'] = pd.to_datetime(stay_df['intime'])

# Unir con eventos de ventilación
vent_times = pe_vent_df.merge(stay_df, on=['subject_id', 'stay_id'])
vent_times['hours_until_vent'] = (vent_times['starttime'] - vent_times['intime']).dt.total_seconds() / 3600

# Calcular la media
avg_hours_until_vent = vent_times['hours_until_vent'].mean()
print(f"Media de horas desde ingreso hasta ventilación: {avg_hours_until_vent:.2f}")


In [ ]:
# Obtener datos de pacientes no ventilados
no_vent_subject_ids = all_df[~all_df['subject_id'].isin(pe_vent_df['subject_id'])]['subject_id'].unique()

# Obtener 'intime' para estos pacientes
no_vent_stays = stay_df[stay_df['subject_id'].isin(no_vent_subject_ids)].copy()

# Calcular la ventana de tiempo para ellos
no_vent_stays['start_window'] = no_vent_stays['intime'] + pd.to_timedelta(avg_hours_until_vent - 6, unit='h')
no_vent_stays['end_window'] = no_vent_stays['intime'] + pd.to_timedelta(avg_hours_until_vent, unit='h')

In [ ]:
# Juntar medidas con las ventanas definidas
no_vent_with_measures = all_df.merge(no_vent_stays[['subject_id', 'stay_id', 'start_window', 'end_window']], on=['subject_id', 'stay_id'])

# Filtrar por la ventana de tiempo definida
no_vent_window_df = no_vent_with_measures[
    (no_vent_with_measures['charttime'] >= no_vent_with_measures['start_window']) &
    (no_vent_with_measures['charttime'] <= no_vent_with_measures['end_window'])
].copy()


In [ ]:
no_vent_df = no_vent_window_df.pivot_table(
    index=['subject_id', 'hadm_id', 'stay_id', 'charttime'],
    columns='itemid',
    values='valuenum',
    aggfunc='first'
).reset_index()

# Renombrar columnas si lo deseas
no_vent_df = no_vent_df.rename(columns=item_names)
no_vent_df.to_csv('no_vent_6h.csv', index=False)

In [ ]:
pd.read_csv('no_vent_6h.csv')

## Limpieza de tablas

### SI ventilación

In [ ]:
df_si_vent = pd.read_csv('si_vent_6h.csv')
df_si_vent.info()

In [ ]:
df_si_vent.describe()

In [ ]:
# Check for missing values
print("\n % Missing values per column:")
print(df_si_vent.isnull().sum()/len(df_si_vent)*100)

In [ ]:
# Drop columns with more than 90% missing values
df_si_vent = df_si_vent.drop(columns=['Temperature_Celsius', 'SaO2', 'HCO3_serum'])

All the columns do have a lot of missing values (more than 40%) so on the first sight we would just drop almost everything but then there is no data left... However, we often have multiple measurements per patient so we can impute within each patient group. In each group, we first fill missing values forward (previous value known is used) followed by filling missing values backward (next known value is used). This method makes sense in clinical time-series data because patient vitals tend to change gradually and also data really is different for every patient.

In [ ]:
# Sort by subject, vent_starttime, and charttime
df_si_vent_sorted = df_si_vent.sort_values(by=['subject_id', 'vent_starttime', 'charttime'])
# Group by patient AND ventilation event
df_si_vent_filled = df_si_vent_sorted.groupby(['subject_id', 'vent_starttime'], group_keys=False).apply(
    lambda group: group.ffill().bfill())
df_si_vent_filled.to_csv('df_si_vent_filled.csv', index=False)
df_si_vent_filled.head()

In [ ]:
print("\n % Missing values per column:")
print(df_si_vent_filled.isnull().sum()/len(df_si_vent_filled)*100)

There is made a huge improvement in the amount of missing values. Heart rate, RR and Spo2 can be imputed safely with the median value of the corresponding column. The other columns will also be imputed at this moment but there need to be looked carefully at importance of these predictors. If they aren't important, we will need to drop them after testing a model.

In [ ]:
# Impute low-missing columns
for col in ['Heart_Rate', 'RR', 'SpO2']:
    df_si_vent_filled[col] = df_si_vent_filled[col].fillna(df_si_vent_filled[col].median())

# Impute high-missing columns temporarily for modeling
for col in ['BP_Systolic', 'BP_Diastolic', 'PCO2_arterial', 'PH_arterial',
            'GCS_Eye_Opening', 'GCS_Verbal_Response', 'GCS_Motor_Response', 'Base_Excess']:
    df_si_vent_filled[col] = df_si_vent_filled[col].fillna(df_si_vent_filled[col].median())

In [ ]:
print("\n % Missing values per column:")
print(df_si_vent_filled.isnull().sum()/len(df_si_vent_filled)*100)

In [ ]:
df_si_vent_filled.to_csv('df_si_vent_filled.csv', index=False)

### NO ventilación

In [ ]:
df_no_vent = pd.read_csv('no_vent_6h.csv')
df_no_vent.info()

In [ ]:
df_no_vent.describe()

In [ ]:
# Check for missing values
print("\n % Missing values per column:")
print(df_no_vent.isnull().sum()/len(df_no_vent)*100)

In [ ]:
# Drop columns with more than 90% missing values (in the patients with vent) bcs to make a model they will need the same amount of colums
df_no_vent = df_no_vent.drop(columns=['Temperature_Celsius', 'SaO2', 'HCO3_serum'])

In [ ]:
# Sort by subject and time
df_no_vent_sorted = df_no_vent.sort_values(by=['subject_id', 'charttime'])

# Group by patient and apply proper fill methods
df_no_vent_filled = df_no_vent_sorted.groupby('subject_id', group_keys=False).apply(
    lambda group: group.ffill().bfill()
)
df_no_vent_filled.head()

In [ ]:
print("\n % Missing values per column:")
print(df_no_vent_filled.isnull().sum()/len(df_no_vent_filled)*100)

In [ ]:
# Impute low-missing columns
for col in ['Heart_Rate', 'RR', 'SpO2', 'GCS_Eye_Opening','GCS_Verbal_Response', 'GCS_Motor_Response']:
    df_no_vent_filled[col] = df_no_vent_filled[col].fillna(df_no_vent_filled[col].median())

# Impute high-missing columns temporarily for modeling
for col in ['BP_Systolic', 'BP_Diastolic', 'PCO2_arterial', 'PH_arterial',
            'Base_Excess']:
    df_no_vent_filled[col] = df_no_vent_filled[col].fillna(df_no_vent_filled[col].median())

df_no_vent_filled.to_csv('df_no_vent_filled.csv', index=False)

In [ ]:
print("\n % Missing values per column:")
print(df_no_vent_filled.isnull().sum()/len(df_no_vent_filled)*100)

## Cálculos tablas

In [ ]:
# Cargar datos
si_vent_df = pd.read_csv("df_si_vent_filled.csv")
no_vent_df = pd.read_csv("df_no_vent_filled.csv")

In [ ]:
# Lista de columnas que no son mediciones (columnas que no deben agregarse)
non_measure_cols = ['subject_id', 'hadm_id', 'stay_id', 'charttime', 'vent_starttime'] if 'vent_starttime' in si_vent_df.columns else ['subject_id', 'hadm_id', 'stay_id', 'charttime']

# Extraer solo las columnas de medidas
value_cols_si = [col for col in si_vent_df.columns if col not in non_measure_cols]
value_cols_no = [col for col in no_vent_df.columns if col not in non_measure_cols]

# Agrupar por paciente y calcular estadísticas
agg_funcs = ['mean', 'min', 'max', 'var']

si_vent_stats = si_vent_df.groupby(['subject_id', 'stay_id'])[value_cols_si].agg(agg_funcs).reset_index()
no_vent_stats = no_vent_df.groupby(['subject_id', 'stay_id'])[value_cols_no].agg(agg_funcs).reset_index()

# Aplanar columnas multi-índice (ej: Heart_Rate_mean, etc.)
si_vent_stats.columns = ['subject_id', 'stay_id'] + [f"{col}_{stat}" for col in value_cols_si for stat in agg_funcs]
no_vent_stats.columns = ['subject_id', 'stay_id'] + [f"{col}_{stat}" for col in value_cols_no for stat in agg_funcs]

# Añadir columna target
si_vent_stats['ventilado'] = 1
no_vent_stats['ventilado'] = 0

In [ ]:
si_vent_stats.to_csv('si_vent_stats.csv', index=False)
si_vent_stats

In [ ]:
no_vent_stats.to_csv('no_vent_stats.csv', index=False)
no_vent_stats